<a href="https://colab.research.google.com/github/QuinnG17/Financial-Programs/blob/main/DAILY_METRIC_CALCULATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import hashlib
import time
import requests
from requests.structures import CaseInsensitiveDict
import pandas as pd
import datetime
from datetime import timedelta
import re
import numpy as np

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

spark_df = spark.sql("SELECT * FROM `hive_metastore`.`alpha_two`.`all_daily_returns`")

previous_metrics = spark.sql("SELECT * FROM `hive_metastore`.`alpha_two`.`all_date_metrics`")
past_metrics = previous_metrics.toPandas()
past_metrics['Date'] = pd.to_datetime(past_metrics['Date'])




In [ ]:
df = spark_df.toPandas()
df = df.set_index('Date')
df = df.sort_index(ascending=True)
past_metrics = past_metrics.sort_values('Date')

df = df.div(100)

metrics_dict = {}
all_output = pd.DataFrame({})

for symbol in df.columns:

    symbol = symbol

    if symbol not in metrics_dict.keys():
        metrics_dict[symbol] = {}

    start_date = past_metrics['Date'].iloc[-1]

    iterations = df[df.index > start_date]

    for date in iterations.index:
        #print(f"{symbol} at {date}")

        time_period = df[df.index < date]

        daily_returns = time_period[symbol]
        #print(daily_returns)

        sharpe_ratio = (252 ** 0.5) * (daily_returns.mean() / daily_returns.std())

        # calculate the Net Profit %
        returns = daily_returns.add(1)

        cumprod = returns.cumprod()

        net_profit_pct = (cumprod.iloc[-1] - 1)*100
        #print(net_profit_pct)

        # calculate the Drawdown %
        equity_curve = cumprod * 100000
        #rint(equity_curve)
        previous_peaks=equity_curve.cummax()
        drawdowns = (equity_curve - previous_peaks) / previous_peaks
        drawdown = drawdowns.min() * 100


        # calculate the Total Trades

        trades = daily_returns[daily_returns != 0]

        trades = len(trades)

        # calculate the Average Win and Average Loss
        gains = daily_returns[daily_returns > 0]
        #rint(len(gains))
        losses = daily_returns[daily_returns < 0]
        #rint(len(losses))
        average_win = gains.mean()
        average_loss = losses.mean() * -1

        # calculate the Alpha
        risk_free_rate = 0.02 / 365
        benchmark_daily_returns = time_period
        benchmark_returns = benchmark_daily_returns['HLT'] #Should be SPY but we don't have data for that
        #benchmark_returns = benchmark_returns.loc[daily_returns.index]
        excess_returns = (daily_returns - benchmark_returns - risk_free_rate)
        #rint(excess_returns)
        beta, alpha = np.polyfit(benchmark_returns, excess_returns, 1)

        # calculate the Compounding Annual Return
        days = (time_period.index[-1] - time_period.index[0]).days
        compound_annual_return = ((1 + (net_profit_pct / 100)) ** (365.25 / days)) - 1

        # calculate the Expectancy
        if trades > 0:
            win_rate = (len(gains) / trades) * 100
            loss_rate = (len(losses) / trades) * 100
            expectancy = (average_win * win_rate / 100) - (average_loss * loss_rate / 100)
        else:
            win_rate = 0
            loss_rate = 0
            expectancy = 0

        # calculate the Beta
        beta = excess_returns.cov(benchmark_returns) / benchmark_returns.var()

        # calculate the Loss Rate
        loss_rate = (losses.count() / trades) * 100

        # calculate the Profit-Loss Ratio
        profit_loss_ratio = gains.sum() / losses.abs().sum()

        # calculate the Annual Standard Deviation and Annual Variance
        annual_std_dev = daily_returns.std() * (252 ** 0.5)
        annual_var = annual_std_dev ** 2

        # calculate the Information Ratio and Tracking Error
        if symbol =='HLT':
            information_ratio = 0
        else:
            information_ratio = excess_returns.mean() / excess_returns.std()

        tracking_error = excess_returns.std()

        # calculate the Total Fees
        total_trades = trades
        commission_per_trade = 5.00
        total_fees = total_trades * commission_per_trade

        # calculate the Treynor Ratio
        # calculate the Treynor Ratio
        beta, alpha = np.polyfit(benchmark_returns, excess_returns, 1)
        risk_free_rate = 0.02
        portfolio_returns = daily_returns.mean()
        treynor_ratio = (portfolio_returns - risk_free_rate) / beta


        metrics_dict[symbol][date] = {
                'Symbol' : symbol,
                'Date': date,
                'Sharpe_Ratio': float(sharpe_ratio),
                'Net_Profit': float(net_profit_pct),
                'Drawdown': float(drawdown),
                'Total_Trades': float(trades),
                'Average_Win': float(average_win),
                'Average_Loss': float(average_loss),
                'Alpha': float(alpha),
                'Compounding_Annual_Return': float(compound_annual_return * 100),
                'Expectancy': float(expectancy),
                'Beta': float(beta),
                'Loss_Rate': float(loss_rate),
                'Win_Rate': float(win_rate),
                'Profit_Loss_Ratio': float(profit_loss_ratio),
                'Annual_Standard_Deviation': float(annual_std_dev),
                'Annual_Variance': float(annual_var),
                'Information_Ratio': float(information_ratio),
                'Tracking_Error': float(tracking_error),
                'Total_Fees': float(total_fees),
                'Treynor_Ratio': float(treynor_ratio)
            }

    print(len(metrics_dict.keys()) / len(df.columns))

In [ ]:
all_output = pd.DataFrame.from_dict(metrics_dict['HLT'])

all_output = all_output.T

for key in metrics_dict.keys():
    symbol_output = pd.DataFrame.from_dict(metrics_dict[key])
    symbol_output = symbol_output.T
    all_output = pd.concat([all_output, symbol_output])

from pyspark.sql import SparkSession
output = all_output

cols_to_convert = output.columns[2:]  # Get all column names except the first 2 columns
output[cols_to_convert] = output[cols_to_convert].astype(float)
output['Symbol'] = output['Symbol'].str.strip()
output['Symbol'] = output['Symbol'].astype(str)


print(output)

In [ ]:
all_data = pd.concat([past_metrics, output])
all_data = all_data.sort_values(by=['Symbol', 'Date'])
print(all_data)

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

# Create a SparkSession
spark = SparkSession.builder.appName('pandas-to-databricks').getOrCreate()

# Convert the Pandas DataFrame to a Databricks DataFrame
db_df = spark.createDataFrame(all_data)

db_df.createOrReplaceTempView('final_output')

db_df.show()

In [ ]:
%sql create or replace table alpha_two.ALL_DATE_METRICS_BACKUP as select * from alpha_two.ALL_DATE_METRICS

In [ ]:
%sql create or replace table alpha_two.ALL_DATE_METRICS as select * from final_output